In [2]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

### Using Dataset that created in last exercise


In [3]:
data=pd.read_csv("Canada_Postal_code.csv")

In [5]:
data.drop("Unnamed: 0",axis=1,inplace=True)

In [7]:
data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [8]:
address='Toronto, Ontario Canada'

geolocator=Nominatim(user_agent="ny_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude

In [10]:
latitude

43.6534817

In [23]:
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(data['Latitude'],data['Longitude'],data['Borough'],data['Neighbourhood']):
    label='{}, {}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
        ).add_to(map_toronto)
    
map_toronto

### Now, I am going to randomly select one neighborhood of Toronto

In [31]:
ran=np.random.randint(0,len(data))
neighbour=data.loc[ran,'Neighbourhood']

In [60]:
data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [32]:
neighbour_latitude=data.loc[ran,'Latitude']
neighbour_longitude=data.loc[ran,'Longitude']

### Foursquare Credentials and Version

In [77]:
client_id='CDC0BC1OFLXFNDMO44XMF3GFZCCOZDUIUD5W0S2KYCZMHHKN'
client_secret='PTYI2XEMPZRKGIQPVHIWVOOGOJ4ZXBTAOEVS1YZ4L1GAK5P5'
version='20180605'
limit=100

In [35]:
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    client_id, 
    client_secret, 
    version, 
    neighbour_latitude, 
    neighbour_longitude, 
    radius, 
    limit)

In [36]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fecad3ad547f51ffbd1e1e7'},
 'response': {'headerLocation': 'Briar Hill - Belgravia',
  'headerFullLocation': 'Briar Hill - Belgravia, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.714077004500005,
    'lng': -79.43885887350775},
   'sw': {'lat': 43.7050769955, 'lng': -79.4512863264922}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4cc75fe53c40a35d09976e2e',
       'name': 'R Bakery - Delicious Cakes, Breads',
       'location': {'address': '326 Marlee Ave',
        'lat': 43.70741964331273,
        'lng': -79.44312584563261,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.70741964331273,
          'lng': -79.44312584563261}],
        'dista

In [37]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
        
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues=results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]

nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)

nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Selvam\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,"R Bakery - Delicious Cakes, Breads",Bakery,43.707420,-79.443126
1,Miyako Sushi Restaurant,Japanese Restaurant,43.709111,-79.443930
2,"Chalker's Pub, Billiards and Bistro",Pub,43.705747,-79.442378
3,Fraserwood Park,Park,43.713550,-79.442482


## Explore Neighbourhoods in Toronto

In [89]:
def getNearbyVenues(names,latitudes,longitudes,radius=500):
    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        print(name)
        
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    client_id, 
    client_secret, 
    version, 
    neighbour_latitude, 
    neighbour_longitude, 
    radius, 
    limit)
            
        results=requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([
            (name,lat,lng,v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results
        ])
        
        nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns= ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        
        return nearby_venues

In [90]:
data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [91]:
toronto=getNearbyVenues(names=data['Neighbourhood'],latitudes=data['Latitude'],longitudes=data['Longitude'])

Parkwoods


In [93]:
toronto.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,"R Bakery - Delicious Cakes, Breads",43.707420,-79.443126,Bakery
1,Parkwoods,43.753259,-79.329656,Miyako Sushi Restaurant,43.709111,-79.443930,Japanese Restaurant
2,Parkwoods,43.753259,-79.329656,"Chalker's Pub, Billiards and Bistro",43.705747,-79.442378,Pub
3,Parkwoods,43.753259,-79.329656,Fraserwood Park,43.713550,-79.442482,Park
